In [1]:
import sys
import requests
import uuid
import json
import time

import openc2


host = 'localhost'
port = '5000'


def get_uuid():
    return str(uuid.uuid4())

def get_headers(uuid):
    headers = {
        "request_id": uuid,
        "created": time.time()
    }
    return headers


def _get_investigate():
    cmd = {
        "action": "investigate",
        "target": {
            "ipv4_connection": {
                "src_addr": "172.17.0.2",
                "src_port": 0,
                "dst_addr": "104.244.42.130",
                "dst_port": 0,
                "protocol": "tcp"
                }
        },
        "args": {
            "huntargs": {},
            "returnvars": ["nt_to_investigate"],
            "response_requested": "complete"
            },
        "actuator": {
            "x-kestrel": {
                "huntbook": "exfil"
            }
        }
    }
    return cmd

def _get_query():
    cmd = {
        "action": "query",
        "target": {"features": ["versions", "profiles", "rate_limit", "pairs"]},
        "args": {},
        "actuator": {
            "x-kestrel": {}
        }
    }
    return cmd

def get_cmd(action):
    cmd = {}
    if action == "query":
        cmd = _get_query()
    elif action == "investigate":
        cmd = _get_investigate()

    oc2cmd = {
        "openc2": {
            "request": cmd
        }
    }
    return oc2cmd

In [2]:
uuid = get_uuid()
headers = {
        "Content-type": "application/openc2+json;version=1.0",
        "Accept": "application/openc2+json;version=1.0",
        "X-Request-ID": uuid
    }
body = {
        "headers": get_headers(uuid),
        "body": get_cmd("query")
    }

res = requests.post('http://'+host+':'+port+'/v1/openc2', data=json.dumps(body), headers=headers)
print(json.dumps(res.json(), indent=4))

response from server: {"body":{"openc2":{"results":{"pairs":{"investigate":[]},"profiles":["kestrel"],"rate_limit":30,"x-kestrel":{"anomaly":{"retvars":["proc_truepositives"],"vars":["source","process.pid"]},"cos":{"retvars":["cos_conn"],"vars":["source","ipv4_connection.dst_addr"]},"exfil":{"retvars":["nt_to_investigate","nt_truepositives"],"vars":["source","ipv4_connection.dst_addr"]},"network":{"retvars":["gpproc"],"vars":["monitored_host","src_ip","src_port","dst_ip","dst_port"]},"recon":{"retvars":["newvar"],"vars":["source"]},"rootcause":{"retvars":["ppp_all"],"vars":["source","process.pid"]}}},"status":200,"status_text":""}},"headers":{"created":1659121127.8046186,"request_id":"00858eba-c088-4d8c-a16d-a7cc45e3ef03"}}

